In [26]:
import nltk

In [27]:
from transformers import pipeline
from transformers import BartForConditionalGeneration, BartTokenizer
from keybert import KeyBERT

In [28]:
import tensorflow as tf
import torch
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import random
import copy
import sys

In [29]:
from nltk import wsd
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from spacy.cli import download
from spacy import load
import warnings
from nltk.tokenize import sent_tokenize
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prita\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [30]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\prita\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [31]:
import spacy
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('wordnet2022')
nlp = spacy.load('en_core_web_sm')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\prita\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prita\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to
[nltk_data]     C:\Users\prita\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet2022 is already up-to-date!


In [32]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prita\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
import requests
import json
import re
import random
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [34]:
model= BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer= BartTokenizer.from_pretrained("facebook/bart-large-cnn")

In [35]:
def summarize(text, maxSummarylength=2000):
    inputs = tokenizer.encode("summarize: " + text,
                              return_tensors="pt",
                              max_length=1024, truncation=True)
    summary_ids = model.generate(
        inputs,
        max_length=int(maxSummarylength),
        min_length=int(maxSummarylength / 5),
        length_penalty=10.0,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [36]:
def split_text_into_pieces(text, max_tokens=3000, overlapPercent=10):
  tokens=tokenizer.tokenize(text)
  overlap_tokens=int(max_tokens * overlapPercent/100)
  pieces=[tokens[i:i + max_tokens]
          for i in range(0, len(tokens),
                         max_tokens - overlap_tokens)]
  text_pieces= [tokenizer.decode(
      tokenizer.convert_tokens_to_ids(piece),
      skip_special_tokens= True) for piece in pieces]

  return text_pieces

In [37]:
def recursive_summarize(text, max_length=2000, recursionLevel=0):
    recursionLevel=recursionLevel+1
    print("######### Recursion level: ",
          recursionLevel,"\n\n######### ")
    tokens = tokenizer.tokenize(text)
    expectedCountOfChunks = len(tokens)/max_length
    max_length=int(len(tokens)/expectedCountOfChunks)+2

    # Break the text into pieces of max_length
    pieces = split_text_into_pieces(text, max_tokens=max_length)

    print("Number of pieces: ", len(pieces))
    # Summarize each piece
    summaries=[]
    k=0
    for k in range(0, len(pieces)):
        piece=pieces[k]
        print("****************************************************")
        print("Piece:",(k+1)," out of ", len(pieces), "pieces")
        print(piece, "\n")
        summary = summarize(piece, maxSummarylength=int(max_length/3*2))
        print("SUMNMARY: ", summary)
        summaries.append(summary)
        print("****************************************************")

    concatenated_summary = ' '.join(summaries)

    tokens = tokenizer.tokenize(concatenated_summary)

    if len(tokens) > max_length:
        # If the concatenated_summary is too long, repeat the process
        print("############# GOING RECURSIVE ##############")
        return recursive_summarize(concatenated_summary,
                                   max_length=max_length,
                                   recursionLevel=recursionLevel)
    else:
      # Concatenate the summaries and summarize again
        final_summary=concatenated_summary
        if len(pieces)>1:
            final_summary = summarize(concatenated_summary,
                                  maxSummarylength=max_length)
        return final_summary

In [38]:
def read_text_from_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print("File not found. Please ensure the path is correct.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [39]:
model_2= KeyBERT(model="distilbert-base-nli-mean-tokens")

In [40]:
%%time
def keyword_extract(article):
    keywords = model_2.extract_keywords(
        article,
        top_n=200,
        keyphrase_ngram_range=(1, 1),  # Can also generate phrases of more than 1 word (1,2), (1,3)
        stop_words="english"
    )
    return keywords


CPU times: total: 0 ns
Wall time: 0 ns


In [41]:
# Automatic POS Tagging & Lesk with spacy

POS_MAP = {
    'VERB': wn.VERB,
    'NOUN': wn.NOUN,
    'PROPN': wn.NOUN
}


def lesk(doc, word):
    found = False
    for token in doc:
        if token.text.lower() == word.lower():
            word = token
            found = True
            break
    if not found:
        #warning.warn(f'Word \"{word}\" does not appear in the document: {doc.text}.')
        return None
    pos = POS_MAP.get(word.pos_, False)
    if not pos:
        #warnings.warn(f'POS tag for {word.text} not found in wordnet. Falling back to default Lesk behaviour.')
        return None
    args = [c.text for c in doc], word.text
    kwargs = dict(pos=pos)
    return wsd.lesk(*args, **kwargs)

In [42]:
def corresponding_sentences(text,filtered_keywords):
  sentences=sent_tokenize(text)      #Sentence tokenization
  corr_sentences=dict()
  for keyword in filtered_keywords:
    for sentence in sentences:
      if keyword.lower() in sentence.lower():
        corr_sentences.update({keyword:sentence})  #if keyword is found in the sentence then the dictionary is updated with the corresponding key value pair
  return(corr_sentences)

In [43]:
def get_wordsense(sent, word):
    word = word.lower()

    if len(word.split()) > 0:
        word = word.replace(" ", "_")

    synsets = wn.synsets(word, 'n')

    if synsets:
        try:
            wup = max_similarity(sent, word, 'wup', pos='n')
            adapted_lesk_output = adapted_lesk(sent, word, pos='n')
            lowest_index = min(synsets.index(wup), synsets.index(adapted_lesk_output))
            return synsets[lowest_index]
        except (IndexError,ValueError):
            return None
    else:
        return None

In [44]:
def get_distractors_wordnet(syn, word):
    distractors = []
    word = word.lower()
    orig_word = word

    if len(word.split()) > 0:
        word = word.replace(" ", "_")

    hypernym = syn.hypernyms()

    if len(hypernym) == 0:
        return distractors

    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()

        if name == orig_word:
            continue

        name = name.replace("_", " ")
        name = " ".join(w.capitalize() for w in name.split())

        if name is not None and name not in distractors:
            distractors.append(name)

    return distractors


In [45]:
def get_distractors_conceptnet(word):
    word = word.lower()
    original_word = word

    if len(word.split()) > 0:
        word = word.replace(" ", "_")

    distractor_list = []
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5" % (word, word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term']

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10" % (link, link)
        obj2 = requests.get(url2).json()

        for edge in obj2['edges']:
            word2 = edge['start']['label']

            if word2 not in distractor_list and original_word.lower() not in word2.lower():
                distractor_list.append(word2)

    return distractor_list

In [46]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name_2 = "t5-small"
tokenizer_2 = T5Tokenizer.from_pretrained(model_name_2)
model_4 = T5ForConditionalGeneration.from_pretrained(model_name_2)


In [47]:
def generate_mcq(keyword_sentences, key_distractor_list):
    index = 1
    for keyword, sentence in keyword_sentences.items():
        if keyword not in key_distractor_list:
            print(f"Skipping '{keyword}': No distractors found.")
            continue

        pattern = re.compile(re.escape(keyword), re.IGNORECASE)

        distractors = key_distractor_list[keyword]
        choices = [keyword.capitalize()] + distractors
        prompt = f"Here is a sentence with a blank: {sentence.replace(keyword, '_______')}. What is the missing word? Options: {', '.join(choices)}"

        input_ids = tokenizer_2(prompt, return_tensors="pt")
        output = model_4.generate(**input_ids)
        generated_answer = tokenizer_2.decode(output[0], skip_special_tokens=True)
        generated_answer = generated_answer.strip()

        masked_sentence = pattern.sub(generated_answer, sentence)
        print(f"{index}) {masked_sentence}")

        top4choices = choices[:4]
        random.shuffle(top4choices)
        optionchoices = ['a', 'b', 'c', 'd']
        for idx, choice in enumerate(top4choices):
            print(f"\t{optionchoices[idx]}) {choice}")
        print("\nMore options:", choices[4:], "\n")
        index += 1


In [48]:
choice=int(input("1 if you want to upload text file\n2 or any number if you want to enter text directly"))
if(choice==1):
  user_file_path = input("Please enter the path to your .txt file: ")
  article = read_text_from_file(user_file_path)
else:
  article=input("Enter the text: ")

summary= recursive_summarize(article)

original_keywords=keyword_extract(article)

required_keywords=[i[0] for i in original_keywords]

filtered_keywords=set()
for keyword in required_keywords:
  if(keyword.lower() in summary.lower()):
    filtered_keywords.add(keyword)

keyword_sentences=corresponding_sentences(summary, filtered_keywords)
key_distractor_list = {}

for keyword, sentence in keyword_sentences.items():
    wordsense = get_wordsense(sentence, keyword)
    if wordsense:
        distractors = get_distractors_wordnet(wordsense, keyword)
        if len(distractors) == 0:
            distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors
    else:
        distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors

generate_mcq(keyword_sentences, key_distractor_list)

1 if you want to upload text file
2 or any number if you want to enter text directly1
Please enter the path to your .txt file: chap_not_afraid_to_die (1).txt
######### Recursion level:  1 

######### 
Number of pieces:  2
****************************************************
Piece: 1  out of  2 pieces

IN July 1976, my wife Mary, son Jonathan, 6, daughter Suzanne,
7, and I set sail from Plymouth, England, to duplicate the roundthe-world voyage made 200 years earlier by Captain James Cook.
For the longest time, Mary and I — a 37-year-old businessman —
had dreamt of sailing in the wake of the famous explorer, and
for the past 16 years we had spent all our leisure time honing
our seafaring skills in British waters.
Our boat Wavewalker, a 23 metre, 30 ton wooden-hulled
beauty, had been professionally built, and we had spent months
fitting it out and testing it in the roughest weather we could find.
The first leg of our planned three-year, 105,000 kilometre
journey passed pleasantly as we sa

SUMNMARY:  In 1976, a crew of four set sail from Plymouth, England, to duplicate the roundthe-world voyage made 200 years earlier by Captain James Cook. The boat, Wavewalker, was a 23 metre, 30 ton wooden-hulled 30 ton beauty. The first leg of their planned three-year, 105,000 kilometrejourney passed pleasantly as we sailed down the west coast of purposefullyAfrica to Cape Town. But on New Year’s Day, the waves were gigantic and we were still making eight knots. A torrent of green and white water broke over the ship, my head smashed into the wheel and I was aware of flying overboard and sinking below the waves. Somehow, I found the wheel, lined up the stern for the next wave and hung on. The ship was near capsizing, her masts almost horizontally. Then a wave hurled her upright, my lifeline jerked into the air and I sailed through the air into Wavewalker’S main boom. Subsequent waves tossed me around the deck like a rag doll. My left ribs cracked; my mouth filled with blood and broken t

8) Subsequent waves tossed me around the deck like a rag True.
	a) Doll
	b) Balloon
	c) Ball
	d) Cockhorse

More options: ['Dollhouse', 'Frisbee', 'Hobby', 'Hula-hoop', 'Jack-in-the-box', 'Jumping Jack', 'Jungle Gym', 'Kaleidoscope', 'Kite', 'Lego', 'Meccano', 'Pea Shooter', 'Pinata', 'Pinwheel', 'Playhouse', 'Pogo Stick', 'Popgun', 'Rattle', 'Sandbox', 'Seesaw', 'Slide', 'Slingshot', 'Stick Horse', 'Swing', 'Teddy', 'Top', 'Train Set', 'Water Pistol', 'Yo-yo'] 

9) The first leg of their planned three-year, 105,000 kilometreTrue passed pleasantly as we sailed down the west coast of purposefullyAfrica to Cape Town.
	a) Circumnavigation
	b) Commutation
	c) Air Travel
	d) Journey

More options: ['Crossing', 'Driving', 'Junketing', 'On The Road', 'Peregrination', 'Riding', 'Stage', 'Staging', 'Traversal', 'Walk', 'Wandering', 'Water Travel', 'Wayfaring'] 

10) The boat, Wavewalker, was a 23 metre, 30 ton wooden-Trueed 30 ton beauty.
	a) Hull
	b) Pappus

More options: [] 

11) Then a wave 

25) ‘‘I’m not afraid to die,’ said a six-year-old boy who was not afraid of aminor blood clot between skin and skull.’ ‘I don’t want you to worry about a head injury’, said a seven-year old girl who did not want us to worry.‘’You’re the best daddy in the whole world — and the best ,,,, Mate, Privateer, Purser, Sailing Master,“ my son replied.” “Not today, Jon, I’ve afraid” said Sue in a matter-of-fact voice.
	a) Privateer
	b) Mate
	c) Master
	d) Captain

More options: ['Purser', 'Sailing Master', 'Steward', 'Supercargo'] 

26) The boat, Wavewalker, was a 23 metre, True ton wooden-hulled True ton beauty.
	a) Billion
	b) Crore
	c) Aleph-null
	d) 30

More options: ['Eighteen', 'Eighty', 'Eleven', 'Fifteen', 'Fifty', 'Five Hundred', 'Forty', 'Fourteen', 'Great Gross', 'Gross', 'Hundred', 'Hundred Thousand', 'Long Hundred', 'Million', 'Nineteen', 'Ninety', 'Octillion', 'Quadrillion', 'Quintillion', 'Septillion', 'Seventeen', 'Seventy-eight', 'Seventy', 'Sextillion', 'Sixteen', 'Sixty', 'Tee

In [50]:
import os
print(os.getcwd())


C:\Users\prita\machine learning and data science
